Warunki startowe Programu :

In [8]:
using Plots , LinearAlgebra
max_i=10000
epsilon = 1e-8
matrix = [4 -1 0; -1 4 -1; 0 -1 4]

3×3 Matrix{Int64}:
  4  -1   0
 -1   4  -1
  0  -1   4

#### Metoda __Potęgowa__

Opis Algorytmu

   a. Inicjalizacja losowego wektora `v`.
   
   b. Normalizacja wektora `v`.
   
   c. Iteracje do maksymalnej liczby iteracji (`max_i`).
   
   d. Obliczenie nowego wektora `v_new` przez pomnożenie macierzy `matrix_c` przez wektor `v`.
   
   e. Obliczenie wartości własnej (`eigen`) jako iloczynu skalarnego wektorów `v` i `v_new`.
   
   f. Normalizacja wektora `v_new`.
   
   g. Sprawdzenie warunku zbieżności. Przerwanie iteracji, jeśli warunek spełniony.
   
   h. Aktualizacja macierzy `matrix_c` odejmując wartość własną pomnożoną przez zewnętrzny iloczyn wektorów (`v * v'`).
   
Jest to implementacja metody __Potęgowej__ w wersji pozwalającej obliczyć każdy wektor własny wraz z wartością, po znalezieniu jednej wartości własnej metoda odejmuje składnik dominujący
`matrix_c -= eigen * (v * v')`
następnie ponownie aplikuje metodę potęgową na zdeflowanej macierzy w celu znalezienia kolejnej wartości własnej. Ten proces jest powtarzany, aż do uzyskania wszystkich wartości własnych.


In [9]:
matrix_c = copy(matrix)
function power(matrix_c,max_i, epsilon)
    n = size(matrix_c, 1)
    for _ in 1:n
        v = randn(n)
        v /= norm(v, 2)
        eigen = 0.0
        for i in 1:max_i
            v_new = matrix_c * v
            eigen = dot(v, v_new)
            v = v_new / norm(v_new, 2)
            if norm(matrix_c * v - eigen * v, 2) < epsilon
                println("Liczba iteracji po których osiągnięto dokładność ",i)
                break
            end
        end
        matrix_c -= eigen * (v * v')
        println("Wartość własna: ",eigen,"\nDla wektora ",v)
    end
end
@time power(matrix_c,max_i,epsilon)


Liczba iteracji po których osiągnięto dokładność 65
Wartość własna: 5.414213562373095
Dla wektora [0.5000000043583002, -0.7071067811865475, 0.4999999956416999]
Liczba iteracji po których osiągnięto dokładność 47
Wartość własna: 4.0
Dla wektora [0.7071067795679989, 9.509410536789886e-9, -0.707106782805096]
Liczba iteracji po których osiągnięto dokładność 2
Wartość własna: 2.585786437626905
Dla wektora [0.49999999441528714, 0.7071067811865476, 0.5000000055847127]


#### Metoda __Rayleigha__

Opis Algorytmu

   a. Inicjalizacja wektora `v` jako wektora jednostkowego.
   
   b. Iteracje do maksymalnej liczby iteracji (`max_i`).
   
   c. Obliczenie wartości własnej (`eigen`) jako iloczynu skalarnego wektora `v` i macierzy `matrix` pomnożonej przez wektor `v`.
   
   d. Aktualizacja wektora własnego (`v`) poprzez pomnożenie macierzy `matrix` przez wektor `v`.
   
   e. Normalizacja wektora `v`.
   
   f. Sprawdzenie warunku zbieżności. Przerwanie iteracji, jeśli warunek spełniony.


In [10]:
matrix_c = copy(matrix)
function rayleigh(matrix, max_i, epsilon)
    n = size(matrix, 1)
    eigen = 0.0
    v = ones(n) / sqrt(n)  
    for i in 1:max_i
        eigen = dot(v, matrix * v)
        v = matrix * v
        v /= norm(v)
        if norm(matrix * v - eigen * v) < epsilon
            println("Liczba iteracji po których osiągnięto dokładność ", i)
            break
        end
    end
    println("Wartość własna: ", eigen, "\nDla wektora ", v)
end 
@time rayleigh(matrix_c, max_i, epsilon)


Liczba iteracji po których osiągnięto dokładność 29
Wartość własna: 5.414213562373095
Dla wektora [0.5000000014360919, -0.7071067791556069, 0.5000000014360919]


#### Metoda __QR__

a. Inicjalizacja macierzy ortogonalnej \(V\) jako macierzy jednostkowej.

   b. Iteracje do maksymalnej liczby iteracji (`max_i`).

   c. Obliczenie rozkładu QR macierzy `matrix_c`, gdzie \(Q\) to macierz ortogonalna, a \(R\) to macierz trójkątna górna.

   d. Aktualizacja macierzy `matrix_c` jako iloczyn \(R i Q\).

   e. Akumulacja macierzy ortogonalnej \(V\) przez mnożenie przez \(Q\).

   f. Sprawdzenie warunku zbieżności. Przerwanie iteracji, jeśli warunek spełniony.

In [11]:
matrix_c = copy(matrix)
function qr_method(matrix_c, max_i, epsilon)
    n = size(matrix_c, 1)
    V = I(n) 
    for i in 1:max_i
        Q, R = qr(matrix_c)
        matrix_c = R * Q
        V *= Q
        if norm(matrix_c - diagm(diag(matrix_c))) < epsilon
            println("Liczba iteracji po których osiągnięto dokładność ",i)
            break
        end
    end
    eigenvalues = diag(matrix_c)
    eigenvectors = V
    return eigenvalues, eigenvectors
end

@time eigenvalues, eigenvectors = qr_method(matrix_c,max_i,epsilon)

for i in 1:size(matrix_c,1)
    println("Wartość własna: ",eigenvalues[i],"\nDla wektora ",eigenvectors[:,i])
end

Liczba iteracji po których osiągnięto dokładność 65
Wartość własna: 5.414213562373096
Dla wektora [-0.5000000028451155, 0.7071067811865502, -0.4999999971548882]
Wartość własna: 4.000000000000001
Dla wektora [-0.7071067791750913, -2.845596956565709e-9, 0.7071067831980081]
Wartość własna: 2.585786437626904
Dla wektora [0.4999999999995166, 0.7071067811865475, 0.5000000000004825]


In [12]:
@time eigenvalues, eigenvectors = eigen(matrix)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 2.585786437626906
 4.000000000000002
 5.414213562373095
vectors:
3×3 Matrix{Float64}:
 0.5        0.707107     -0.5
 0.707107  -7.85046e-16   0.707107
 0.5       -0.707107     -0.5